In [1]:
from glob import glob
import pandas as pd
import os
import math
from tqdm import tqdm
import seaborn as sns


PROFILE_PATH = "single_nuclei_gaussian3_normalized_all_compounds.csv"
METADATA_PATH = "/home/christer/Datasets/MCF7_paper/metadata/filtered_metadata.csv"

In [2]:
df_profiles = pd.read_csv(PROFILE_PATH, index_col=0)
metadata = pd.read_csv(METADATA_PATH, index_col=0)

In [3]:
# Merge the two dataframes df_profiles and metadata
profiles = df_profiles.set_index("path").join(
    metadata.set_index("Image_FileName_DAPI")[["Image_Metadata_Plate_DAPI",
                                               "Image_Metadata_Compound",
                                               "Image_Metadata_Concentration"]])
profiles = profiles.reset_index()

In [4]:
controls = profiles[profiles["Image_Metadata_Compound"] == "DMSO"].drop(
    columns=["path", "Image_Metadata_Compound", "Image_Metadata_Concentration"]).groupby("Image_Metadata_Plate_DAPI").mean()

In [18]:
distances = None
for idx_row, row in tqdm(profiles.iterrows(), total=profiles.shape[0]):
    ctrl_features = controls.loc[row["Image_Metadata_Plate_DAPI"]]
    row_features = row.drop(labels=["Image_Metadata_Compound", "Image_Metadata_Concentration", "path", "Image_Metadata_Plate_DAPI"])

    diff = ctrl_features - row_features
    diff["path"] = row["path"]
    diff["Image_Metadata_Plate_DAPI"] = row["Image_Metadata_Plate_DAPI"]
    
    if not isinstance(distances, pd.DataFrame):
        distances = pd.DataFrame(columns=diff.index.to_list())
    distances.loc[idx_row] = diff

  1%|          | 4962/927657 [07:00<21:43:49, 11.79it/s]

KeyboardInterrupt



In [ ]:
tqdm.pandas()

def calculate_diff(row, controls):
    ctrl_features = controls.loc[row["Image_Metadata_Plate_DAPI"]]
    row_features = row.drop(
        labels=["Image_Metadata_Compound", "Image_Metadata_Concentration", "path", "Image_Metadata_Plate_DAPI"])
    diff = ctrl_features - row_features
    diff["path"] = row["path"]
    diff["Image_Metadata_Plate_DAPI"] = row["Image_Metadata_Plate_DAPI"]
    return diff

distances = profiles.progress_apply(calculate_diff, axis=1, args=(controls,))

100%|██████████| 927657/927657 [20:50<00:00, 635.54it/s]  